# 변수 설명 및 대회 전략
+ GitHub README에 다 적어놨습니다.
+ 중복데이터 처리를 어떻게 해야할지 고민을 하였습니다.
+ Feature Engineering보단 Catboost의 Categorical 데이터에 관한 하이퍼파라미터 설정에 포커스를 맞췄습니다.
+ [GitHub](https://github.com/ds-wook/PredictCreditCardDelinquency)

# 참고한 자료
+ [Hayo 님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2519?page=1&dtype=recent)
+ [rollcake님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2526?page=1&dtype=recent)
+ [datu님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2515?page=2&dtype=recent)
+ [catboost 논문](https://arxiv.org/pdf/1706.09516.pdf)
+ [catboost 활용 논문](https://arxiv.org/abs/2104.07553)

In [3]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'
!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

ERROR: Invalid requirement: '#'


  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to c:\users\public\documents\estsoft\creatortemp\pip-install-t1uratdz\pytorch-tabnet
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-3.1.1-py3-none-any.whl size=39905 sha256=9ccc35e67f1f2d9ef4ad019987dade1349d2007168da94d2c4855d338cc8d7b6
  Stored in directory: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-ephem-wheel-cache-w8hmpgov\wheels\00\61\ad\ba7b2cff3afdf0971f6205bd564b90aaab96bf84c8cc36ef9e
Successfully built pytorch-tabnet
  Attempting uninstall: pytorch-tabnet
    Found existing installation: pytorch-tabnet 3.1.1
    Uninstalling pytorch-tabnet-3.1.1:
      Successfully uninstall

In [7]:
!pip install catboost
!pip install lightgbm

In [8]:
from typing import Dict, Tuple, Union, List

import numpy as np
import pandas as pd
import torch

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

C:\Users\hwang in beom\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


# Load Dataset
+ catboost가 이 대회의 핵심 알고리즘이라고 생각하여 categorical한 변수를 만드는게 중요하다고 생각하여 따로 함수를 만듬
+ 다른 알고리즘들은 categorical한 변수보다 numeric한 변수 처리에 초점을 맞춤

In [19]:
def category_income(data: pd.DataFrame) -> pd.DataFrame:
    """
    2.객체간의 비교 연산 메소드
      eq() == 같다 / ne() =! 다르다 / lt() < 작다 / gt() > 크다 / le() < 작거나 같다 / ge() >= 크거나 같다
      기호와 메서드의 차이 => 메서드에서 제공하는 옵션을 사용해서 좀 더 정교한 연산을 수행한다.
    """
    
    data["income_total"] = data["income_total"] / 10000
    conditions = [
        (data["income_total"].le(18)),
        (data["income_total"].gt(18) & data["income_total"].le(33)),
        (data["income_total"].gt(33) & data["income_total"].le(49)),
        (data["income_total"].gt(49) & data["income_total"].le(64)),
        (data["income_total"].gt(64) & data["income_total"].le(80)),
        (data["income_total"].gt(80) & data["income_total"].le(95)),
        (data["income_total"].gt(95) & data["income_total"].le(111)),
        (data["income_total"].gt(111) & data["income_total"].le(126)),
        (data["income_total"].gt(126) & data["income_total"].le(142)),
        (data["income_total"].gt(142)),
    ]
    choices = [i for i in range(10)]

    data["income_total"] = np.select(conditions, choices)
    return data


def load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
    path = "C:/Users/hwang in beom/Desktop/data/1.신용카드 연체/"
    train = pd.read_csv(path + "train.csv")
    train = train.drop(["index"], axis=1)
    train.fillna("NAN", inplace=True)

    test = pd.read_csv(path + "test.csv")
    test = test.drop(["index"], axis=1)
    test.fillna("NAN", inplace=True)

    """
    3.절대값  + 음수 제거
    """
    # absolute 
    train["DAYS_EMPLOYED"] = train["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    train["DAYS_EMPLOYED"] = np.abs(train["DAYS_EMPLOYED"])
    test["DAYS_EMPLOYED"] = test["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    test["DAYS_EMPLOYED"] = np.abs(test["DAYS_EMPLOYED"])
    train["DAYS_BIRTH"] = np.abs(train["DAYS_BIRTH"])
    test["DAYS_BIRTH"] = np.abs(test["DAYS_BIRTH"])
    train["begin_month"] = np.abs(train["begin_month"]).astype(int)
    test["begin_month"] = np.abs(test["begin_month"]).astype(int)

    """
    4.파생 변수 생성 - DAYS_BIRTH 값 month / week 
    """
    # DAYS_BIRTH
    train["DAYS_BIRTH_month"] = np.floor(train["DAYS_BIRTH"] / 30) - (
        (np.floor(train["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_BIRTH_month"] = train["DAYS_BIRTH_month"].astype(int)
    train["DAYS_BIRTH_week"] = np.floor(train["DAYS_BIRTH"] / 7) - (
        (np.floor(train["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_BIRTH_week"] = train["DAYS_BIRTH_week"].astype(int)
    test["DAYS_BIRTH_month"] = np.floor(test["DAYS_BIRTH"] / 30) - (
        (np.floor(test["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_BIRTH_month"] = test["DAYS_BIRTH_month"].astype(int)
    test["DAYS_BIRTH_week"] = np.floor(test["DAYS_BIRTH"] / 7) - (
        (np.floor(test["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_BIRTH_week"] = test["DAYS_BIRTH_week"].astype(int)

    
    """
    5.Age 변수 날짜를 나이로 변환  
    """
    # Age
    train["Age"] = np.abs(train["DAYS_BIRTH"]) // 360
    test["Age"] = np.abs(test["DAYS_BIRTH"]) // 360
    
    
    """
    6.DAYS_EMPLOYED_month 변수 -> month / week 라는 파생변수 생성
    """
    # DAYS_EMPLOYED
    train["DAYS_EMPLOYED_month"] = np.floor(train["DAYS_EMPLOYED"] / 30) - (
        (np.floor(train["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_EMPLOYED_month"] = train["DAYS_EMPLOYED_month"].astype(int)
    train["DAYS_EMPLOYED_week"] = np.floor(train["DAYS_EMPLOYED"] / 7) - (
        (np.floor(train["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_EMPLOYED_week"] = train["DAYS_EMPLOYED_week"].astype(int)
    test["DAYS_EMPLOYED_month"] = np.floor(test["DAYS_EMPLOYED"] / 30) - (
        (np.floor(test["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_EMPLOYED_month"] = test["DAYS_EMPLOYED_month"].astype(int)
    test["DAYS_EMPLOYED_week"] = np.floor(test["DAYS_EMPLOYED"] / 7) - (
        (np.floor(test["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_EMPLOYED_week"] = test["DAYS_EMPLOYED_week"].astype(int)
    
    
    """
    7.EMPLOYED 는 연도로 나눔
    """
    # EMPLOYED
    train["EMPLOYED"] = train["DAYS_EMPLOYED"] / 360
    test["EMPLOYED"] = test["DAYS_EMPLOYED"] / 360


    """
    8.before_EMPLOYED 라는 파생 변수 생성 
    태어난 날짜 - 일한날짜 = 일을 안했던 기간을 산정
    그후 이걸 month와 week라는 파생변수 생성
    """
    # before_EMPLOYED
    train["before_EMPLOYED"] = train["DAYS_BIRTH"] - train["DAYS_EMPLOYED"]
    train["before_EMPLOYED_month"] = np.floor(train["before_EMPLOYED"] / 30) - (
        (np.floor(train["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["before_EMPLOYED_month"] = train["before_EMPLOYED_month"].astype(int)
    train["before_EMPLOYED_week"] = np.floor(train["before_EMPLOYED"] / 7) - (
        (np.floor(train["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["before_EMPLOYED_week"] = train["before_EMPLOYED_week"].astype(int)
    test["before_EMPLOYED"] = test["DAYS_BIRTH"] - test["DAYS_EMPLOYED"]
    test["before_EMPLOYED_month"] = np.floor(test["before_EMPLOYED"] / 30) - (
        (np.floor(test["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["before_EMPLOYED_month"] = test["before_EMPLOYED_month"].astype(int)
    test["before_EMPLOYED_week"] = np.floor(test["before_EMPLOYED"] / 7) - (
        (np.floor(test["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["before_EMPLOYED_week"] = test["before_EMPLOYED_week"].astype(int)

    """
    9.user_code 생성 
    gender + car + reality 
    """
    
    # gender_car_reality
    train["user_code"] = (
        train["gender"].astype(str)
        + "_"
        + train["car"].astype(str)
        + "_"
        + train["reality"].astype(str)
    )
    test["user_code"] = (
        test["gender"].astype(str)
        + "_"
        + test["car"].astype(str)
        + "_"
        + test["reality"].astype(str)
    )

    del_cols = [
        "gender",
        "car",
        "reality",
        "email",
        "child_num",
        "DAYS_BIRTH",
        "DAYS_EMPLOYED",
    ]
    
    """
    10.family_size 이상치 처리
    """
    train.drop(train.loc[train["family_size"] > 7, "family_size"].index, inplace=True)
    train.drop(del_cols, axis=1, inplace=True)
    test.drop(del_cols, axis=1, inplace=True)

    cat_cols = [
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "occyp_type",
        "user_code",
    ]

    """
    11.label encoder 
    """
    for col in cat_cols:
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(train[col])
        train[col] = label_encoder.transform(train[col])
        test[col] = label_encoder.transform(test[col])

    return train, test


def cat_load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    # 1. finllna 및 필요없는 index 삭제
    path = "C:/Users/hwang in beom/Desktop/data/1.신용카드 연체/"
    train = pd.read_csv(path + "train.csv")
    train = train.drop(["index"], axis=1)
    train.fillna("NAN", inplace=True)

    test = pd.read_csv(path + "test.csv")
    test = test.drop(["index"], axis=1)
    test.fillna("NAN", inplace=True)


# 	2. 절대값  + 음수 제거
    # absolute
    train["DAYS_EMPLOYED"] = train["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    train["DAYS_EMPLOYED"] = np.abs(train["DAYS_EMPLOYED"])
    test["DAYS_EMPLOYED"] = test["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    test["DAYS_EMPLOYED"] = np.abs(test["DAYS_EMPLOYED"])
    train["DAYS_BIRTH"] = np.abs(train["DAYS_BIRTH"])
    test["DAYS_BIRTH"] = np.abs(test["DAYS_BIRTH"])
    train["begin_month"] = np.abs(train["begin_month"]).astype(int)
    test["begin_month"] = np.abs(test["begin_month"]).astype(int)

    
# 3. 객체간의 비교 연산 메소드
#   eq() == 같다 / ne() =! 다르다 / lt() < 작다 / gt() > 크다 / le() < 작거나 같다 / ge() >=크거나 같다.
#  기호와 메서드의 차이 => 메서드에서 제공하는 옵션을 사용해서 좀 더 정교한 연산을 수행한다.
    # income_total
    train = category_income(train)
    test = category_income(test)

    
    
# 	4. 파생 변수 생성 - DAYS_BIRTH 값 month / week

    # DAYS_BIRTH
    train["DAYS_BIRTH_month"] = np.floor(train["DAYS_BIRTH"] / 30) - (
        (np.floor(train["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_BIRTH_month"] = train["DAYS_BIRTH_month"].astype(int)
    train["DAYS_BIRTH_week"] = np.floor(train["DAYS_BIRTH"] / 7) - (
        (np.floor(train["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_BIRTH_week"] = train["DAYS_BIRTH_week"].astype(int)
    test["DAYS_BIRTH_month"] = np.floor(test["DAYS_BIRTH"] / 30) - (
        (np.floor(test["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_BIRTH_month"] = test["DAYS_BIRTH_month"].astype(int)
    test["DAYS_BIRTH_week"] = np.floor(test["DAYS_BIRTH"] / 7) - (
        (np.floor(test["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_BIRTH_week"] = test["DAYS_BIRTH_week"].astype(int)

    
# 	5. Age 변수 날짜를 나이로 변환  
    # Age
    train["Age"] = np.abs(train["DAYS_BIRTH"]) // 360
    test["Age"] = np.abs(test["DAYS_BIRTH"]) // 360

# 	6. DAYS_EMPLOYED_month 변수 -> month / week 라는 파생변수 생성
    # DAYS_EMPLOYED
    train["DAYS_EMPLOYED_month"] = np.floor(train["DAYS_EMPLOYED"] / 30) - (
        (np.floor(train["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_EMPLOYED_month"] = train["DAYS_EMPLOYED_month"].astype(int)
    train["DAYS_EMPLOYED_week"] = np.floor(train["DAYS_EMPLOYED"] / 7) - (
        (np.floor(train["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_EMPLOYED_week"] = train["DAYS_EMPLOYED_week"].astype(int)
    test["DAYS_EMPLOYED_month"] = np.floor(test["DAYS_EMPLOYED"] / 30) - (
        (np.floor(test["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_EMPLOYED_month"] = test["DAYS_EMPLOYED_month"].astype(int)
    test["DAYS_EMPLOYED_week"] = np.floor(test["DAYS_EMPLOYED"] / 7) - (
        (np.floor(test["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_EMPLOYED_week"] = test["DAYS_EMPLOYED_week"].astype(int)

    

# 	7. EMPLOYED 는 연도로 나눔
    # EMPLOYED
    train["EMPLOYED"] = train["DAYS_EMPLOYED"] / 360
    test["EMPLOYED"] = test["DAYS_EMPLOYED"] / 360

    
    
# 	8. before_EMPLOYED 라는 파생 변수 생성
# 태어난 날짜 - 일한날짜 = 일을 안했던 기간을 산정
#  그 후 이걸 month와 week라는 파생변수 생성
    # before_EMPLOYED 
    train["before_EMPLOYED"] = train["DAYS_BIRTH"] - train["DAYS_EMPLOYED"]
    train["before_EMPLOYED_month"] = np.floor(train["before_EMPLOYED"] / 30) - (
        (np.floor(train["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["before_EMPLOYED_month"] = train["before_EMPLOYED_month"].astype(int)
    train["before_EMPLOYED_week"] = np.floor(train["before_EMPLOYED"] / 7) - (
        (np.floor(train["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["before_EMPLOYED_week"] = train["before_EMPLOYED_week"].astype(int)
    test["before_EMPLOYED"] = test["DAYS_BIRTH"] - test["DAYS_EMPLOYED"]
    test["before_EMPLOYED_month"] = np.floor(test["before_EMPLOYED"] / 30) - (
        (np.floor(test["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["before_EMPLOYED_month"] = test["before_EMPLOYED_month"].astype(int)
    test["before_EMPLOYED_week"] = np.floor(test["before_EMPLOYED"] / 7) - (
        (np.floor(test["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["before_EMPLOYED_week"] = test["before_EMPLOYED_week"].astype(int)

    
# 	9. user_code 생성
#gender + car + reality

    # gender_car_reality
    train["user_code"] = (
        train["gender"].astype(str)
        + "_"
        + train["car"].astype(str)
        + "_"
        + train["reality"].astype(str)
    )
    test["user_code"] = (
        test["gender"].astype(str)
        + "_"
        + test["car"].astype(str)
        + "_"
        + test["reality"].astype(str)
    )

    del_cols = [
        "gender",
        "car",
        "reality",
        "email",
        "child_num",
        "DAYS_BIRTH",
        "DAYS_EMPLOYED",
    ]

# 	11. family_size 이상치 처리
    train.drop(train.loc[train["family_size"] > 7, "family_size"].index, inplace=True)
    train.drop(del_cols, axis=1, inplace=True)
    test.drop(del_cols, axis=1, inplace=True)

    cat_cols = [
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "occyp_type",
        "user_code",
    ]
    
    

# 	12. label encoder
    for col in cat_cols:
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(train[col])
        train[col] = label_encoder.transform(train[col])
        test[col] = label_encoder.transform(test[col])

    return train, test

# Model

In [20]:
# 함수받는 형태가 신기헀다. 이렇게 강제로 받을 수 있는 것 같다.

# Catboost
def stratified_kfold_cat(
    params: Dict[str, Union[int, float, str, List[str]]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    
    #StratifiedKFold 사용
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    #데이터 프레임 만드는 작업(0으로 채워넣어서 만드는 작업)
    cat_oof = np.zeros((X.shape[0], 3))
    cat_preds = np.zeros((X_test.shape[0], 3))
    cat_cols = [c for c in X.columns if X[c].dtypes == "int64"]

    
    cat_cols
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        model = CatBoostClassifier(**params)

        # 100번까지만 돌리고 가장 좋은 모델 true
        model.fit(
            train_data,
            eval_set=valid_data,
            early_stopping_rounds=100,
            use_best_model=True,
            verbose=100,
        )

        cat_oof[valid_idx] = model.predict_proba(X_valid)
        cat_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, cat_oof)
    print(f"Log Loss Score: {log_score:.5f}\n")
    return cat_oof, cat_preds


# Light GBM
def stratified_kfold_lgbm(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    lgb_oof = np.zeros((X.shape[0], 3))
    lgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        pre_model = LGBMClassifier(**params)

        pre_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )
        params2 = params.copy()
        params2["learning_rate"] = params["learning_rate"] * 0.1

        model = LGBMClassifier(**params2)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
            init_model=pre_model,
        )
        lgb_oof[valid_idx] = model.predict_proba(X_valid)
        lgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, lgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return lgb_oof, lgb_preds


# XGB
def stratified_kfold_xgb(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    xgb_oof = np.zeros((X.shape[0], 3))
    xgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )

        xgb_oof[valid_idx] = model.predict_proba(X_valid)
        xgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, xgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return xgb_oof, xgb_preds


# Random Foreset
def stratified_kfold_rf(
    params: Dict[str, Union[int, float, str, bool]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    rf_oof = np.zeros((X.shape[0], 3))
    rf_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        model = RandomForestClassifier(**params)
        model.fit(
            X_train,
            y_train,
        )

        rf_oof[valid_idx] = model.predict_proba(X_valid)
        rf_preds += model.predict_proba(X_test) / n_fold
        print(f"Log Loss Score: {log_loss(y_valid, rf_oof[valid_idx]):.5f}")

    log_score = log_loss(y, rf_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return rf_oof, rf_preds

# Cat Train
+ catboost 같은 경우 categorical한 feature를 고정시켜주는게 핵심이라고 생각하여 하이퍼파라미터튜닝에 신경을 씀
+ 하이퍼파라미터튜닝은 optuna 라이브러리로 했음

In [21]:
train_cat, test_cat = cat_load_dataset()
X = train_cat.drop("credit", axis=1)
y = train_cat["credit"]

X_test = test_cat.copy()

In [23]:
cat_params = {
    "learning_rate": 0.026612467217016746,
    "l2_leaf_reg": 0.3753065117824262,
    "max_depth": 8,
    "bagging_temperature": 1,
    "min_data_in_leaf": 57,
    "max_bin": 494,
    "random_state": 42,
    "eval_metric": "MultiClass",
    "loss_function": "MultiClass",
    "od_type": "Iter",
    "od_wait": 500,
    "iterations": 10000,
    "cat_features": [
        "income_total",
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "FLAG_MOBIL",
        "work_phone",
        "phone",
        "occyp_type",
        "begin_month",
        "DAYS_BIRTH_month",
        "DAYS_BIRTH_week",
        "Age",
        "DAYS_EMPLOYED_month",
        "DAYS_EMPLOYED_week",
        "before_EMPLOYED",
        "before_EMPLOYED_month",
        "before_EMPLOYED_week",
        "user_code",
    ],
}

cat_oof, cat_preds = stratified_kfold_cat(cat_params, 10, X, y, X_test)

============ Fold 0 ============



CatBoostError: categorical features indices in the model are set to [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20] and train dataset categorical features indices are set to [5, 6, 7, 13, 17]

# LGBM Train
+ 하이퍼파라미터 튜닝은 optuna를 사용했음

In [ ]:
train, test = load_dataset()
X = train.drop("credit", axis=1)
y = train["credit"]
X_test = test.copy()

In [ ]:
lgb_params = {
    "reg_alpha": 5.998770177220496e-05,
    "reg_lambda": 0.07127674208132959,
    "max_depth": 18,
    "num_leaves": 125,
    "colsample_bytree": 0.4241631237880101,
    "subsample": 0.8876057928391585,
    "subsample_freq": 5,
    "min_child_samples": 5,
    "max_bin": 449,
    "random_state": 42,
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "n_estimators": 10000,
    "objective": "multiclass",
    "metric": "multi_logloss",
}
lgbm_oof, lgbm_preds = stratified_kfold_lgbm(lgb_params, 10, X, y, X_test)

# XGB Train
+ 하이퍼파라미터 튜닝을 optuna를 사용했음

In [ ]:
xgb_params = {
    "eta": 0.023839252347297356,
    "reg_alpha": 6.99554614267605e-06,
    "reg_lambda": 0.010419988953061583,
    "max_depth": 15,
    "max_leaves": 159,
    "colsample_bytree": 0.4515469593932409,
    "subsample": 0.7732694309118915,
    "min_child_weight": 5,
    "gamma": 0.6847131315687576,
    "random_state": 42,
    "n_estimators": 10000,
    "objective": "multi:softmax",
    "eval_metric": "mlogloss",
}
xgb_oof, xgb_preds = stratified_kfold_xgb(xgb_params, 10, X, y, X_test)

# RandomForest Train

In [ ]:
rf_params = {
        "criterion": "gini",
        "n_estimators": 300,
        "min_samples_split": 10,
        "min_samples_leaf": 2,
        "max_features": "auto",
        "oob_score": True,
        "random_state": 42,
        "n_jobs": -1,
    }
rf_oof, rf_preds = stratified_kfold_rf(rf_params, 10, X, y, X_test)

In [ ]:
train, test = load_dataset()
train_x = train.drop("credit", axis = 1)
train_y = train['credit'].values

In [ ]:
train_pred = np.concatenate([cat_oof, lgbm_oof, xgb_oof, rf_oof], axis=1)
train_pred.shape

In [ ]:
test_pred = np.concatenate([cat_preds, lgbm_preds, xgb_preds, rf_preds], axis=1)
test_pred.shape

# Pytorch Tabular
+ Stacking Ensemble을 사용하며 학습 진행

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
n_fold = 10
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
splits = folds.split(train_pred, train_y)
net_oof = np.zeros((train_pred.shape[0], 3))
net_preds = np.zeros((test_pred.shape[0], 3))
for fold, (train_idx, valid_idx) in enumerate(splits):
    print(f"============ Fold {fold} ============\n")
    X_train, X_valid = train_pred[train_idx], train_pred[valid_idx]
    y_train, y_valid = train_y[train_idx], train_y[valid_idx]
    model = TabNetMultiTaskClassifier(
            n_d=64, n_a=64, n_steps=1,
            lambda_sparse=1e-4,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_params = {"gamma": 0.9, "step_size": 50},
            scheduler_fn=torch.optim.lr_scheduler.StepLR,
            mask_type="entmax", 
            device_name=device
    )

    model.fit(
        X_train, y_train.reshape(-1,1),
        eval_set=[(X_valid, y_valid.reshape(-1,1))],
        max_epochs=100,
        batch_size=1024,
        eval_metric=["logloss"],
        virtual_batch_size=128,
        num_workers=1,
        drop_last=False
    )
    net_oof[valid_idx] = model.predict_proba(X_valid)
    net_preds += model.predict_proba(test_pred)[0] / n_fold
log_score = log_loss(train_y, net_oof)
print(f"Log Loss Score: {log_score:.5f}")

In [ ]:
submission = pd.read_csv("/kaggle/input/predictcreditcarddelinquency/sample_submission.csv")
submission.iloc[:, 1:] = net_preds
submission.to_csv("meta_ensemble_submit.csv", index=False)